To complete this assignment, fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`, as well as your name below.

To make sure everything runs as expected, do the following
- **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart)
- **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

A good introduction to Jupyter notebooks is [here](https://realpython.com/jupyter-notebook-introduction/).


In [ ]:
NAME = ""

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Data-structures" data-toc-modified-id="Data-structures-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data structures</a></span></li><li><span><a href="#Parser-states-and-actions" data-toc-modified-id="Parser-states-and-actions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parser states and actions</a></span></li><li><span><a href="#Training-an-action-classifier" data-toc-modified-id="Training-an-action-classifier-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training an action classifier</a></span><ul class="toc-item"><li><span><a href="#Generate-training-data" data-toc-modified-id="Generate-training-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Generate training data</a></span></li><li><span><a href="#Compute-features" data-toc-modified-id="Compute-features-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Compute features</a></span></li><li><span><a href="#Fit-classifier" data-toc-modified-id="Fit-classifier-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Fit classifier</a></span></li></ul></li><li><span><a href="#Using-the-classifier-to-parse" data-toc-modified-id="Using-the-classifier-to-parse-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Using the classifier to parse</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation</a></span></li><li><span><a href="#Bonus:-Feature-engineering" data-toc-modified-id="Bonus:-Feature-engineering-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Bonus: Feature engineering</a></span></li></ul></div>

## Introduction


In this assignment we will implement a transition-based dependency parser, using machine learning to choose the best transition at each step.

As in the previous assignment, there are spaces below for you to both write code and short answers. In some places, there are tests to check your work, though passing tests does not guarantee full credit. I recommend moving sequentially from top to bottom, getting each step working before moving on to the next.

This assignment will use a number of Python libraries, listed in the next cell. If you haven't already installed these, you can do so by running this command in this directory: `pip install -r requirements.txt`. Minor variants in the version numbers shouldn't affect things much.



In [ ]:
# imports
from collections import Counter, defaultdict, deque
import numpy as np
from numpy import array as npa
import os
import math
import re
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

## Data structures

First, we need a data structure to store dependency trees. We'll use the one below. For simplicity, we'll assume unlabeled dependency trees.

Note that we need the word position in the sentence to deal with the case where a word appears more than once.

The children are stored as a list of other DTree objects.

In [ ]:
class DTree:
    def __init__(self, word, position):
        self.word = word
        self.position = position
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        
    def __repr__(self):
        # for pretty printing a single node
        return '%s-%d' % (self.word, self.position)
    
    def print(self, indent=0):
        # for printing a node and all its descendents
        print(' '*indent + '-'*min(2, indent) + str(self))
        for c in self.children:
            c.print(indent + 2)

Here's an example tree for our example "Book me the morning flight"

In [ ]:
def example_tree():
    root = DTree('ROOT', 0)
    book = DTree('Book', 1)
    me = DTree('me', 2)
    the = DTree('the', 3)
    morning = DTree('morning', 4)
    flight = DTree('flight', 5)

    root.add_child(book)
    book.add_child(me)
    book.add_child(flight)
    flight.add_child(the)
    flight.add_child(morning)
    return root

root = example_tree()
root.print()


Output should be:
    
```
ROOT-0
  --Book-1
    --me-2
    --flight-5
      --the-3
      --morning-4
```

Given a `DTree` defined above, we will often want to extract the parents of each word. Complete the method below, which takes in a DTree and returns a `dict` that maps each `DTree` node to its parent node.

In [ ]:
def get_parents(node, node2parent):
    """
    Create a dict of string->string indicating who the parent of each node is in 
    a dependency tree.
    
    Args:
      node..........a DTree node
      node2parent...a dict from str to str, where the value is the parent of the key.
      You can use str(node) to get the string representation of each node (word-position),
      which should be used for both keys and values
      
    Returns:
      nothing. node2parent is passed by reference and is used as the result, to make it 
      easier to implement this recursively.
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

node2parent = {}
get_parents(example_tree(), node2parent)

Expected output:

```
{'Book-1': 'ROOT-0',
 'me-2': 'Book-1',
 'flight-5': 'Book-1',
 'the-3': 'flight-5',
 'morning-4': 'flight-5'}
```

In [ ]:
# test tree: ROOT -> c1 -> c2
root = DTree('ROOT', 0)
c1 = DTree('C1', 1)
c2 = DTree('C2', 2)
root.add_child(c1)
c1.add_child(c2)

node2parent_test = {}
get_parents(root, node2parent_test)

assert node2parent_test[str(c1)] == str(root)
assert node2parent_test[str(c2)] == str(c1)

## Parser states and actions

Next, we need a method to construct a dependency tree given a sentence and a list of dependency parse operators. Recall that the operators are:

- **SHIFT**: move a word from the input buffer to the stack
- **ARC_L**: left arc; add a dependency link from the last element of the stack to the second-to-last element of the stack. remove the second-to-last element of the stack.
- **ARC_R**: right arc; add a dependency link from the second-to-last element of the stack to the last element of the stack. remove the last element of the stack.

e.g., 

**Input:**
- ['book', 'me', 'the', 'morning', 'flight']
- ['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R']

**Output:**

The tree

```
ROOT
  --Book
    --me
    --flight
      --the
      --morning
```

To handle this, we'll use an additional data structure that keeps track of the parser state.

We'll rely on the [deque](https://docs.python.org/3/library/collections.html#collections.deque) class, which is a double-ended queue for efficient insertion and removal from the head/tail of a list.


In [ ]:
# example of deque usage.

stack = deque() 
# push elements onto stack
stack.append('a')
stack.append('b')
stack.append('c')
 
print('Initial stack:')
print(stack)

print('\nWe can pop the last element with .pop()')
print(stack.pop())
print('leaving')
print(stack)

print('\nLike a normal list, we can "peek" at any element we like without removing it')
print('peek', stack[-1])
print('stack is still')
print(stack)
# print(stack.pop())
# print(stack.pop())

print('\nsince it is double-ended, we can also use it like a queue and pop from the left:')
print(stack.popleft())
print('this will be useful for storing words in the sentence')

In [ ]:
class State:
    """
    stack....stack of DTree nodes (dequeue)
    input....input DTree nodes (dequeue)
    root.....reference to the ROOT DTree node
    
    Note that we initialize the State with a ROOT node, then 
    push the words in the sentence onto the input deque as DTree objects.
    """
    def __init__(self, sentence):
        self.stack = deque()
        self.stack.append(DTree('ROOT', 0))
        self.input = deque([DTree(w, i) for i, w in enumerate(sentence)])
        self.root = self.stack[0]        

    def __repr__(self):
        return 'input: %s' % str([str(i) for i in self.input]) + \
               '\nstack: %s' % str([str(i) for i in self.stack])

# we'll use this example sentence a lot.
EXAMPLE_SENTENCE = ['Book', 'me', 'the', 'morning', 'flight']
state = State(EXAMPLE_SENTENCE)
state

With the `State` class, we can now implement `apply_action`. This function will modify the state based on the provided action. We'll assume the action is a string in `['SHIFT', 'ARC_L', 'ARC_R']`.

In [ ]:
def apply_action(state, action):
    """
    Modify the state based on the provided action.
    Args:
      state....a State object
      action...a string in ['SHIFT', 'ARC_L', 'ARC_R']
    Returns:
      nothing. state is modified.
    """
    if action == 'SHIFT':
        # YOUR CODE HERE
        raise NotImplementedError()
    elif action == 'ARC_L':
        # YOUR CODE HERE
        raise NotImplementedError()
    elif action == 'ARC_R':
        # YOUR CODE HERE
        raise NotImplementedError()
    else:
        raise Exception('Unknown action: %s' % action)
    

state = State(EXAMPLE_SENTENCE)
print('Initial state')
print(state)

print('\nDo SHIFT')
apply_action(state, 'SHIFT')
print(state)

print('\nDo SHIFT')
apply_action(state, 'SHIFT')
print(state)

print('\nDo ARC_L')
apply_action(state, 'ARC_L')
print(state)

print('\nDo ARC_R')
apply_action(state, 'ARC_R')
print(state)

print('\ncurrent tree')
state.root.print()

Expected output:
    
```
Initial state
input: ['Book-0', 'me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']

Do SHIFT
input: ['me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0']

Do SHIFT
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0', 'me-1']

Do ARC_L
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'me-1']

Do ARC_R
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']

current tree
ROOT-0
  --me-1
    --Book-0
```

In [ ]:
# Test on our running example.
EXAMPLE_ACTIONS = ['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R']
state_test = State(EXAMPLE_SENTENCE)
nodes_test = state_test.input.copy()

for action in EXAMPLE_ACTIONS:
    apply_action(state_test, action)  

# test that the final tree looks like this:
# ROOT-0
#   --Book-1
#     --me-2
#     --flight-5
#       --the-3
#       --morning-4

assert state_test.root.word == 'ROOT'
book = state_test.root.children[0]
assert book  == nodes_test[0]

me = book.children[0]
assert me == nodes_test[1]

flight = book.children[1]
assert flight == nodes_test[4]

# note that "the" is the first child of flight and morning is the second, due to the order in which they
# are added.
the = flight.children[1]
assert the == nodes_test[2]

morning = flight.children[0]
assert morning == nodes_test[3]

We will need a way to determine when there are no more actions left to perform. I.e., when should we stop calling apply_action? What properties should the State have when we should stop? Implement the method below to do so.

In [ ]:
def is_finished(state):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
state_test = State(EXAMPLE_SENTENCE)
nodes_test = state_test.input.copy()

# apply all but the last action and confirm is_finished is False
for action in EXAMPLE_ACTIONS[:-1]:
    apply_action(state_test, action)  
    assert is_finished(state_test) == False

# apply final action
apply_action(state_test, EXAMPLE_ACTIONS[-1])
assert is_finished(state_test) == True

Below is a helper function to apply all the actions to a given sentence.

In [ ]:
def apply_actions(sentence, actions, verbose=False):
    state = State(sentence)
    for action in actions:

        if is_finished(state): # stop if needed
            break
            
        apply_action(state, action)
        if verbose:
            print('\nstate after action %s' % action)
            print(state)
    return state

# note that in the final answer, "morning" and "the" are swapped due to the order the dependencies are added.
final_state = apply_actions(EXAMPLE_SENTENCE, EXAMPLE_ACTIONS, verbose=True)

print('\nfinal tree:')
final_state.root.print()

One thing we've ignored above is that some actions may be impossible. For example, we can't shift when there are no words left in the input. Likewise, we can't do `ARC_L` or `ARC_R` if there are fewer than two elements on the stack.

Implement the method below to check for these cases. The method will return a valid action string if the given action is invalid. 

If `SHIFT` is illegal, return `ARC_R`.

If `ARC_L` or `ARC_R` is illegal, return `SHIFT`.

If no moves are legal, raise an exception `raise Exception('no valid action`)`.

In [ ]:
def check_action(state, action):
    # YOUR CODE HERE
    raise NotImplementedError()
        
state = State(['Hi', 'there'])
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
apply_action(state, 'SHIFT')
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
apply_action(state, 'SHIFT')
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
apply_action(state, 'ARC_R')
print(state)


In [ ]:
state = State(['Hi', 'there'])
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
assert check_action(state, 'SHIFT') == 'SHIFT'
apply_action(state, 'SHIFT')
print(state)


print('\ncheck_action', check_action(state, 'SHIFT'))
assert check_action(state, 'SHIFT') == 'SHIFT'
apply_action(state, 'SHIFT')
print(state)


print('\ncheck_action', check_action(state, 'SHIFT'))
assert check_action(state, 'SHIFT') == 'ARC_R'
apply_action(state, 'ARC_R')
print(state)



## Training an action classifier 

OK!  We now have some infrastructure in place to apply a sequence of actions to create a dependency tree.

Next, we'll need to train a classifier to predict the best action at each time step. To do this, we'll use some labeled data. Each example contains a sentence and the sequence of true actions to get the correct dependency tree.


In [ ]:
def read_data(fname):
    data = []    
    for line in open(fname):
        sentence, actions = line.split("\t")
        sentence = sentence.strip().split()
        actions = actions.strip().split()
        data.append({'sentence': sentence, 'actions': actions})
    return data

train_data = read_data(os.path.join('data', 'train.txt'))

print('read %d sentences' % len(train_data))

print('here is the fifth example')
train_data[4]

In [ ]:
# and here we can use apply_actions apply the true actions to get the final dependency tree
final_state = apply_actions(train_data[4]['sentence'], train_data[4]['actions'], verbose=False)
final_state.root.print()

### Generate training data

To generate training examples, we will loop over each example, then apply each true action.

At each step, we will yield a `(state, action)` tuple indicating what the correct action is for that state.

The `state` will be used to compute features, and the `action` will become our class label.

In [ ]:
def generate_training_samples(data):
    for d in tqdm(data):
        sentence = d['sentence'].copy()
        actions = d['actions'].copy()
        state = State(sentence)
        for action in actions:
            # we use yield here so this becomes an iterator
            # this can save memory, as opposed to constructing a 
            # giant list before compute the feature vectors.
            yield(state, action)
            apply_action(state, action)

# here is the first training sample generated
state, action = next(generate_training_samples(train_data))
print('state:')
print(state)
print('action:')
print(action)

### Compute features

While we iterate over the training samples, we will compute a list of features over each state.

A feature will be a tuple of (`feature_name, feature_value`). For most features, the `feature_value` will be 1.0.

For example, here is a feature representing the word at the top of the stack (position -1)

We use the special token `EMPTY` to indicate that there is no word present at the given position.

In [ ]:
def stack_feature(state):
    if len(state.stack) == 0:
        return ('STACK@-1=EMPTY', 1.0)
    else:
        return ('STACK@-1=%s' % state.stack[-1].word, 1.0)
    
stack_feature(state)

Similarly, here is a feature representing the word at the front of the input (position 0)

In [ ]:
def input_feature(state):
    if len(state.input) == 0:
        return ('INPUT@0=EMPTY', 1.0)
    else:
        return ('INPUT@0=%s' % state.input[0].word, 1.0)
    
input_feature(state)

We can now write a function that iterates over all the training samples generated by `generate_training_samples`, computes all feature functions, and stores the result in a list. Along the way, we'll also keep track of the true actions corresponding to each feature list.

Below, we test this for the first two examples in the train_data. As you can see, even with just two sentences, this generates a lot of training examples.

In [ ]:
def compute_features(sample_iter, feature_fns):
    features = []
    labels = []
    for state, action in sample_iter:
        features.append([feature_fn(state) for feature_fn in feature_fns])
        labels.append(action)
    return features, labels


feature_fns = [stack_feature, input_feature]
features, labels = compute_features(generate_training_samples(train_data[:2]),
                                    feature_fns)
display(list(zip(labels, features)))

Let's write a few more feature functions. Complete the methods below, again using `EMPTY` to indicate that there is no word present at the given position.

In [ ]:
def stack_feature_2(state):
    """
    Return a feature for the second-to-last element of the stack.
    e.g., (STACK@-2=years, 1.0)
    """
    # YOUR CODE HERE
    raise NotImplementedError()

def input_feature_2(state):
    """
    Return a feature for the second element of the input.
    e.g., (INPUT@1=years, 1.0)
    """
    # YOUR CODE HERE
    raise NotImplementedError()

def stack_pair_feature(state):
    """
    A combination of the last and second-to-last elements of the stack.
    e.g., STACK@-1=being__STACK@-2=was
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
state = State(EXAMPLE_SENTENCE)
print(state)
print(stack_feature(state))
assert stack_feature(state) == ('STACK@-1=ROOT', 1.0)
print(stack_feature_2(state))
assert stack_feature_2(state) == ('STACK@-2=EMPTY', 1.0)
print(input_feature(state))
assert input_feature(state) == ('INPUT@0=Book', 1.0)
print(input_feature_2(state))
assert input_feature_2(state) == ('INPUT@1=me', 1.0)
print(stack_pair_feature(state))
assert stack_pair_feature(state) == ('STACK@-1=ROOT__STACK@-2=EMPTY', 1.0)
apply_action(state, 'SHIFT')

print()
print('state after SHIFT')
print(state)
print(stack_feature(state))
print(stack_feature_2(state))
print(input_feature(state))
print(input_feature_2(state))
print(stack_pair_feature(state))
assert stack_pair_feature(state) == ('STACK@-1=Book__STACK@-2=ROOT', 1.0)

In [ ]:
# here are the new features for the first 5 instances.
feature_fns = [stack_feature, input_feature, stack_feature_2, input_feature_2, stack_pair_feature]
features, labels = compute_features(generate_training_samples(train_data[:2]),
                                    feature_fns)
display(list(zip(labels, features))[:5])

Expected output:

```
[('SHIFT',
  [('STACK@-1=ROOT', 1.0),
   ('INPUT@0=Al', 1.0),
   ('STACK@-2=EMPTY', 1.0),
   ('INPUT@1=-', 1.0),
   ('STACK@-1=ROOT__STACK@-2=EMPTY', 1)]),
 ('SHIFT',
  [('STACK@-1=Al', 1.0),
   ('INPUT@0=-', 1.0),
   ('STACK@-2=ROOT', 1.0),
   ('INPUT@1=Zaman', 1.0),
   ('STACK@-1=Al__STACK@-2=ROOT', 1)]),
 ('ARC_R',
  [('STACK@-1=-', 1.0),
   ('INPUT@0=Zaman', 1.0),
   ('STACK@-2=Al', 1.0),
   ('INPUT@1=:', 1.0),
   ('STACK@-1=-__STACK@-2=Al', 1)]),
 ('SHIFT',
  [('STACK@-1=Al', 1.0),
   ('INPUT@0=Zaman', 1.0),
   ('STACK@-2=ROOT', 1.0),
   ('INPUT@1=:', 1.0),
   ('STACK@-1=Al__STACK@-2=ROOT', 1)]),
 ('ARC_R',
  [('STACK@-1=Zaman', 1.0),
   ('INPUT@0=:', 1.0),
   ('STACK@-2=Al', 1.0),
   ('INPUT@1=American', 1.0),
   ('STACK@-1=Zaman__STACK@-2=Al', 1)])]
```

### Fit classifier

To train the classifier, we'll first compute all the features, then use [DictVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) to create a sparse feature matrix.

In [ ]:
features, labels = compute_features(
    generate_training_samples(train_data),
    [stack_feature, stack_feature_2, stack_pair_feature, input_feature, input_feature_2])
vec = DictVectorizer()
X_train = vec.fit_transform(dict(f) for f in features)
X_train.shape

In [ ]:
clf = LogisticRegression(max_iter=100, class_weight='balanced')
clf.fit(X_train, labels)

In [ ]:
# print top coefficients for each class.
feats = npa(vec.get_feature_names())
for ci, cname in enumerate(clf.classes_):
    print(cname)
    theta = clf.coef_[ci]
    for i in np.argsort(theta)[::-1][:15]:
        print(feats[i], theta[i])

**SHORT ANSWER**: 

**Look at the top coefficients for each class. Do any of the top features make sense to you? Pick two features and explain why you think they are highly weighted, along with examples of each. Enter your answer below.**

YOUR ANSWER HERE

## Using the classifier to parse

With the classifier `clf`, we can now predict a good action to take for any possible state.

To do so, we'll create a new class called `ClassifierParser`, which will apply the lcassifier to parse a given sentence.

For any given state, we'll need to (i) compute the feature functions, (ii) transform the feature functions into a sparse vector, (iii) call `clf.predict` to get the most probable action for the given state.

In [ ]:
class ClassifierParser:
    def __init__(self, clf, vec, feature_fns):
        self.clf = clf
        self.vec = vec
        self.feature_fns = feature_fns
        
    def compute_features(self, state):
        """
        Apply all of the `feature_fns` to this state, then
        call `vec.transform` to convert them into a csr_matrix with a single row.
        Return this matrix.
        """
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict_action(self, state):  
        """
        First, call `compute_features` to get the feature matrix for this single instance.
        Then, return the predicted action for this sequence.
        The returned value should be a single string in ['SHIFT', 'ARC_L', 'ARC_R']
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def parse(self, sentence, verbose=False):
        """
        Putting it all together, here we will:
        - Create an initial State for this sentence
        - while the state is not finished:
          + call `predict_action` to get the next action to take
          + call `check_action` to ensure the action is valid
          + apply the action
        
        Return: a tuple:
          state.....the final State at the end of parsing
          actions...the list of actions taken
        """
        # YOUR CODE HERE
        raise NotImplementedError()
    
# we don't get this example exactly right
parser = ClassifierParser(clf, vec, feature_fns)
state, actions = parser.parse(EXAMPLE_SENTENCE)
print('\npredicted actions')
print(actions)
print('\ntrue actions')
print(EXAMPLE_ACTIONS)

print('\npredicted tree')
state.root.print()
print('\ntrue tree')
apply_actions(EXAMPLE_SENTENCE, EXAMPLE_ACTIONS, verbose=False).root.print()

Expected output (some variations expected given slightly different classifier parameters)

```
predicted actions
['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'SHIFT', 'ARC_R', 'ARC_R']

true actions
['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R']

predicted tree
ROOT-0
  --morning-3
    --the-2
    --Book-0
      --me-1
    --flight-4

true tree
ROOT-0
  --Book-0
    --me-1
    --flight-4
      --morning-3
      --the-2
```

In [ ]:
state, actions = parser.parse(train_data[4]['sentence'])
print('\npredicted actions')
print(actions)
print('\ntrue actions')
print(train_data[4]['actions'])

print('\npredicted tree')
state.root.print()
print('\ntrue tree')
apply_actions(train_data[4]['sentence'], train_data[4]['actions'], verbose=False).root.print()

## Evaluation

To measure the accuracy of the parsers, we will first read in the test data, then generate parses for each sentence.
Using our `get_parents` method from above, we'll then compare the parent links in the predicted and true trees.

Our measure will be the fraction of predicted parent links that are present in the true tree.

E.g., if the predicted tree contains edges a->b, b->c, c->d, but the true tree has edges a->b, b->c, b->d, then the score will be 2/3.

We'll compute the accuracy by iterating through every tree and summing together the numerator and denominator.


In [ ]:
test_data = read_data(os.path.join('data', 'test.txt'))
print('read %d test sentences' % len(test_data))

In [ ]:
def evaluate(parser, test_data):
    """
    Evaluate the parser on this test data using the instructions above.
    Return:
      a float indicating the edge accuracy.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

evaluate(parser, test_data)
# I get around 0.58

In [ ]:
# create an oracle classifier for testing the evaluation method on the first test example.
# it should have an accuracy of 1.0, since it just applies the true actions.
class OracleParser:
    def __init__(self, true_actions):
        self.true_actions = true_actions
        
    def parse(self, sentence):
        return apply_actions(sentence, self.true_actions), self.true_actions       
        
assert evaluate(OracleParser(test_data[0]['actions']), test_data[:1]) == 1.0


## Bonus: Feature engineering

We now have a working dependency parser. The accuracy is actually not too bad, given how few features we're using.

For 5 extra points, spend some time on feature engineering and parameter tuning to see if you can improve accuracy. You may want to iterate on a somewhat smaller version of the dataset to save time before applying to the larger dataset. Explain what you did, how much of a difference it made, and why you think it helped.



YOUR ANSWER HERE